In [ ]:
from pynq import Overlay
overlay = Overlay("loop_dma_system_design.bit")

In [ ]:
overlay?

## Allocate PS Memory Buffers

In [ ]:
from pynq import allocate
import numpy as np

data_size = 1_000_000
input_buffer = allocate(shape=(data_size,), dtype=np.uint32) # ~4MB

In [ ]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

### Print first few values of buffer

In [ ]:
for i in range(10):
    print(hex(input_buffer[i]))

In [ ]:
dma = overlay.axi_dma
dma.sendchannel.transfer(input_buffer)

## Read From DMA

In [ ]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

### Initiate the read

In [ ]:
dma.recvchannel.transfer(output_buffer)

### Check the result

In [ ]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

## Verify that the arrays are equal

In [ ]:
print('Arrays are equal: {}'.format(np.array_equal(input_buffer, output_buffer)))

## DMA Register Map

In [ ]:
dma.register_map

### Trigger an error

In [ ]:
dma.recvchannel.transfer(output_buffer)

In [ ]:
dma.register_map

In [ ]:
hex(input_buffer.physical_address)

In [ ]:
hex(output_buffer.physical_address)

## Free Memory Buffers

In [ ]:
del input_buffer, output_buffer